## Estimating likelihoods for a test and training set using a single cross validation replicate

This is some example code to estimate the likelihood for a test set given the posterior from a training set using the output from BEAST2, and R to calculate likelihoods and do some basic data manipulation.

- We first load the phangorn library in R.

In [45]:
library(phangorn)

- We then need to load the data. This is a simulation replicate from the original study where we compare three clock models; the strict (SC), uncorrelated lognormal (ULCN), and uncorrelated exponential (UCED). The complete data are in *s7_exp.fasta*. They were simulated under the UCED model and the JC substitution model. For simplicity, the first half of the alignment was used as the training set, and the second half as a test set. Load the data and slit it into training and test set as follows:

In [46]:
complete_data <- read.dna('s7_exp.fasta', format = 'fasta')
training_set <- phyDat(complete_data[, 1:(ncol(complete_data) / 2)])
test_set <- phyDat(complete_data[, (ncol(complete_data) / 2):ncol(complete_data)])

- The data were analysed using the three clock models to generate phylogenetic trees in BEAST2. The raw trees from beast are chronograms, which include the branch rates in the annotnations. However, we need phylograms to compute the phylogenetic likelihood. To obtain the phylograms, use something like [dendropy](http://dendropy.org) to multiply the branch rates and times, such that the branch lengths correspond to the expected number of substitutions per site. This has already been done for this example. Load the three sets of phylograms with the following code. Each contains 100 phylograms, which is convenient for this example, in practice, however, using about 1,000 to 10,000 might be appropriate:

In [21]:
strict_phylogs <- read.tree('s7_exp_subset_strict_phylogs.trees')
ucln_phylogs <- read.tree('s7_exp_subset_ucld_phylogs.trees')
uced_phylogs <- read.tree('s7_exp_subset_exp_phylogs.trees')

- The next step is to calcluate the phylogenetic likelihood of the test set for each phylogram using phangorn. Note that we did not load the log files from beast. This is because we do not need the subsitution model parameters, since the data were generated under the JC, which is also the default substitution model in phangorn. 

In [26]:
model_likelihoods <- matrix(NA, 100,  3)
colnames(model_likelihoods) <- c('sc', 'ucln', 'uced')
for(i in 1:nrow(model_likelihoods)){
    model_likelihoods[i, 'sc'] <- pml(tree = strict_phylogs[[i]], data = test_set)$logLik
    model_likelihoods[i, 'ucln'] <- pml(tree = ucln_phylogs[[i]], data = test_set)$logLik
    model_likelihoods[i, 'uced'] <- pml(tree = uced_phylogs[[i]], data = test_set)$logLik
}

- The matrix *model_likelihoods* includes the likelihood for each tree under each model. We then compute the mean of each column. In this case the optimal model is the UCED, which matches that used to generate the data:

In [47]:
colMeans(model_likelihoods)

sc      ucln      uced 
-48024.40 -46486.61 -46485.67

- Note that the UCLN and UCED models have similar performance. In this example, the UCED has higher likelihood than the ULCN, but the difference is less than one log unit. In practice, these models are difficult to distinguish. On the contrary, distinguishing between the strict and any of the relaxed clock models is much easier. For example, the strict clock in this example has a likelihood over 1,000 log units lower than either of the relaxed clock models. Another important consideration is that analyses of empirical data should conduct several cross-validation replicates to reduce sampling error.